# 오늘학교 사이트 크롤링
https://academy.prompie.com/schools/school-district/list/

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os


def extract_data(span):
    """강력한 태그로부터 텍스트를 추출합니다."""
    return span.strong.text.strip() if span and span.strong else ''

def scrape_data():
    url = "https://academy.prompie.com/schools/school-district/list/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    # 데이터 저장을 위한 리스트
    school_districts = []
    snu_admission_rates = []
    college_admission_rates = []
    special_high_school_admission_rates = []
    academy_counts = []

    # 상위 30개 학군 처리
    for item in soup.select('span.font-weight-semi-bold'):
        school_districts.append(item.text.strip())
        data_div = item.find_parent('h5').find_next_sibling('div')
        data_items = data_div.find_all('span', class_='font-weight-normal')
        
        # 각 데이터 추출
        snu_admission_rates.append(data_items[0].strong.text.strip())
        college_admission_rates.append(data_items[1].strong.text.strip())
        special_high_school_admission_rates.append(data_items[2].strong.text.strip())
        academy_counts.append(data_items[3].strong.text.strip())

        
    # 31위부터 100위까지 처리
    table_body = soup.find('tbody', class_='main-table-body')
    rows = table_body.find_all('tr')  

    for row in rows[0:70]:  # Adjust the slice as necessary
        rank_cell = row.find('h6', class_='font-weight-normal')
        rank = int(rank_cell.text) if rank_cell else None  
        cells = row.find_all('td')

        if len(cells) == 6:  
            school_districts.append(cells[1].get_text(strip=True))
            snu_admission_rates.append(cells[2].get_text(strip=True))
            college_admission_rates.append(cells[3].get_text(strip=True))
            special_high_school_admission_rates.append(cells[4].get_text(strip=True))
            academy_counts.append(cells[5].get_text(strip=True))



    # 데이터프레임 생성 및 파일 저장
    df = pd.DataFrame({
        '학군지': school_districts,
        '서울대입학률': snu_admission_rates,
        '4년제입학률': college_admission_rates,
        '특목/자사고진학률': special_high_school_admission_rates,
        '학원수': academy_counts
    })

    excel_file = 'school_data1.xlsx'
    
    try:
        df.to_excel(excel_file, index=False)
    except PermissionError:
        print(f"The file {excel_file} is open. Please close it before running the script.")
        return

    print(df.head())


In [23]:
scrape_data()

          학군지 서울대입학률  4년제입학률 특목/자사고진학률     학원수
0  서울 강남구 대치동  3.75%  96.66%    27.13%  1,442개
1  서울 서초구 반포동  4.20%  95.07%    36.02%    456개
2  부산 금정구 구서동  1.75%  96.16%    29.66%    266개
3   서울 양천구 목동  2.41%  90.68%    24.65%  1,022개
4  부산 해운대구 우동  0.99%  94.35%    18.75%    306개
File is closed and ready to be overwritten.


# 분석 시작

In [3]:
import pandas as pd

df = pd.read_excel("school_data.xlsx")
df.head()

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수
0,서울 강남구 대치동,3.75%,96.66%,27.13%,"1,442개"
1,서울 서초구 반포동,4.20%,95.07%,36.02%,456개
2,부산 금정구 구서동,1.75%,96.16%,29.66%,266개
3,서울 양천구 목동,2.41%,90.68%,24.65%,"1,022개"
4,부산 해운대구 우동,0.99%,94.35%,18.75%,306개


In [4]:
df['서울대입학률'] = df['서울대입학률'].str.rstrip('%').astype(float)
df['4년제입학률'] = df['4년제입학률'].str.rstrip('%').astype(float)
df['특목/자사고진학률'] = df['특목/자사고진학률'].str.rstrip('%').astype(float)
df['학원수'] = df['학원수'].str.replace(',', '').str.rstrip('개').astype(int)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   학군지        100 non-null    object 
 1   서울대입학률     100 non-null    float64
 2   4년제입학률     100 non-null    float64
 3   특목/자사고진학률  100 non-null    float64
 4   학원수        100 non-null    int32  
dtypes: float64(3), int32(1), object(1)
memory usage: 3.6+ KB


In [6]:
df.head()

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수
0,서울 강남구 대치동,3.75,96.66,27.13,1442
1,서울 서초구 반포동,4.20,95.07,36.02,456
2,부산 금정구 구서동,1.75,96.16,29.66,266
3,서울 양천구 목동,2.41,90.68,24.65,1022
4,부산 해운대구 우동,0.99,94.35,18.75,306


In [7]:
# 띄어쓰기가 세 개 이상인 경우 처리
def extract_location(row):
    parts = row['학군지'].split(' ')
    if len(parts) >= 3:
        return parts[0], parts[1], ' '.join(parts[2:])
    elif len(parts) == 2:
        return parts[0], parts[1], ''
    else:
        return '', parts[0], ''

# apply 함수를 사용하여 시, 구, 동 열 생성
df['시'], df['구'], df['동'] = zip(*df.apply(extract_location, axis=1))

In [8]:
df.tail(20)

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
80,경기 군포시 산본동,1.05,64.38,10.31,411,경기,군포시,산본동
81,서울 동대문구 장안동,0.52,70.63,13.02,197,서울,동대문구,장안동
82,인천 남동구 논현동,0.91,70.51,7.00,280,인천,남동구,논현동
83,서울 금천구 시흥동,0.91,66.16,11.23,205,서울,금천구,시흥동
84,대전 유성구 상대동,0.92,88.59,11.90,40,대전,유성구,상대동
85,인천 부평구 부평동,0.86,73.10,6.19,219,인천,부평구,부평동
86,서울 마포구 아현동,0.57,79.79,18.08,69,서울,마포구,아현동
87,전남 무안군 삼향읍,0.24,88.82,5.90,163,전남,무안군,삼향읍
88,경기 용인시 기흥구 중동,0.30,83.67,6.08,179,경기,용인시,기흥구 중동
89,서울 도봉구 창동,0.50,66.78,16.44,271,서울,도봉구,창동


In [9]:
# 주소 분할 함수
def split_address(row):
    parts = row.split() # 띄어쓰기 기준으로 분할
    num_parts = len(parts)

    if num_parts >= 4 and '시' in parts[1]:  # 단어가 4개 이상이고, 두 번째 단어에 "시"가 있는 경우
        city = parts[0] + ' ' + parts[1]
        district = parts[2]
        neighborhood = parts[3]
    elif num_parts == 3:  # 단어가 3개인 경우
        city, district, neighborhood = parts
    else:
        city, district, neighborhood = '', '', ''

    return pd.Series([city, district, neighborhood])

# 새로운 열 추가
df[['시', '구', '동']] = df['학군지'].apply(split_address)

df

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
0,서울 강남구 대치동,3.75,96.66,27.13,1442,서울,강남구,대치동
1,서울 서초구 반포동,4.20,95.07,36.02,456,서울,서초구,반포동
2,부산 금정구 구서동,1.75,96.16,29.66,266,부산,금정구,구서동
3,서울 양천구 목동,2.41,90.68,24.65,1022,서울,양천구,목동
4,부산 해운대구 우동,0.99,94.35,18.75,306,부산,해운대구,우동
...,...,...,...,...,...,...,...,...
95,경기 의정부시 녹양동,1.74,87.89,6.19,52,경기,의정부시,녹양동
96,경기 고양시 일산서구 대화동,0.57,73.56,8.81,146,경기 고양시,일산서구,대화동
97,경기 구리시 교문동,0.86,74.86,10.64,85,경기,구리시,교문동
98,전북 전주시 완산구 효자동2가,0.61,87.71,3.70,163,전북 전주시,완산구,효자동2가


In [10]:
city_df = df.groupby('시')[['서울대입학률', '4년제입학률', '특목/자사고진학률']].mean().round(1)
city_df

,서울대입학률,4년제입학률,특목/자사고진학률
시,,,
경기,1.6,80.0,7.6
경기 고양시,2.8,84.1,10.5
경기 성남시,2.3,85.8,10.9
경기 수원시,2.1,89.3,5.8
경기 안양시,0.7,74.8,10.4
경기 용인시,1.1,79.2,7.0
경남,0.9,83.2,14.5
경북,4.0,100.0,20.6
광주,0.6,90.2,8.1


In [11]:
# 각 비율별로 내림차순 정렬하여 순위 확인
sorted_seoul_univ = city_df.sort_values(by='서울대입학률', ascending=False)
sorted_four_year_univ = city_df.sort_values(by='4년제입학률', ascending=False)
sorted_special_high = city_df.sort_values(by='특목/자사고진학률', ascending=False)

## 순위 확인

In [12]:
print("-----서울대 입학률 순위-----")
sorted_seoul_univ

-----서울대 입학률 순위-----


,서울대입학률,4년제입학률,특목/자사고진학률
시,,,
경북,4.0,100.0,20.6
전북 전주시,3.2,90.8,4.0
경기 고양시,2.8,84.1,10.5
충남,2.6,98.6,42.6
경기 성남시,2.3,85.8,10.9
경기 수원시,2.1,89.3,5.8
서울,2.1,80.0,20.3
경기,1.6,80.0,7.6
인천,1.5,76.5,13.0


In [13]:
경북 = df[df['시'] =="경북"]
경북

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
32,경북 김천시 부곡동,4.03,100.0,20.55,45,경북,김천시,부곡동


### 경북 김천시 부곡동
- KT김천지사, 한국전기안전공사 경북서부지사
- 김천서부초등학교,김천부곡초등학교, 김천중학교
- 김천고등학교[3] ->  경북에 포항제철고등학교와 더불어 단 둘뿐인 전국단위 자율형 사립고등학교
- 대우건설 김천 푸르지오 더 퍼스트 : 2024년 6월 입주예정.

In [14]:
전주 = df[df['시'] =="전북 전주시"]
전주

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
46,전북 전주시 완산구 효자동1가,8.02,100.00,3.28,195,전북 전주시,완산구,효자동1가
58,전북 전주시 완산구 서신동,0.87,84.67,5.16,233,전북 전주시,완산구,서신동
98,전북 전주시 완산구 효자동2가,0.61,87.71,3.70,163,전북 전주시,완산구,효자동2가


### 전북 전주시 완산구
- 효자동1가에 상산고 있음. 
- 상산고 가까이에 힐스테이트 어울림 효자(2022년 6월 입주) 있음. 
- 전북의 큰 공기관들이 대거 있음: 전라북도청,전라북도의회,전라북도 소방본부,완산구청,전라북도교육청,전라북도경찰청,한국토지주택공사 전북본부,한국농어촌공사 전북본부,전북지방우정청

- 서신동은 전라북도의 교육 1번지로 정읍시나 익산시 등지에서 학원을 여기까지 오기도 한다.
    - 한일고 있음. 

- 효자동2가
    - 전일고, 호남제일고, 동암고    

In [15]:
print("-----4년제 입학률 순위-----")
sorted_four_year_univ

-----4년제 입학률 순위-----


,서울대입학률,4년제입학률,특목/자사고진학률
시,,,
경북,4.0,100.0,20.6
충남,2.6,98.6,42.6
부산,1.0,92.7,11.4
전남,0.4,92.0,6.7
전북 전주시,3.2,90.8,4.0
광주,0.6,90.2,8.1
대구,1.4,89.4,6.3
경기 수원시,2.1,89.3,5.8
경기 성남시,2.3,85.8,10.9


In [16]:
부산 = df[df['시'] =="부산"]
부산

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
2,부산 금정구 구서동,1.75,96.16,29.66,266,부산,금정구,구서동
4,부산 해운대구 우동,0.99,94.35,18.75,306,부산,해운대구,우동
15,부산 부산진구 당감동,1.18,90.11,20.48,110,부산,부산진구,당감동
21,부산 연제구 연산동,1.49,94.46,5.71,307,부산,연제구,연산동
22,부산 해운대구 재송동,0.41,97.20,16.76,174,부산,해운대구,재송동
23,부산 남구 대연동,0.79,94.65,7.04,281,부산,남구,대연동
36,부산 동래구 사직동,0.40,88.47,8.76,274,부산,동래구,사직동
39,부산 금정구 부곡동,0.79,91.90,7.08,139,부산,금정구,부곡동
47,부산 연제구 거제동,0.37,88.64,8.86,211,부산,연제구,거제동
52,부산 사하구 괴정동,1.57,91.55,4.87,152,부산,사하구,괴정동


In [17]:
전남 = df[df['시'] =="전남"]
전남

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
40,전남 목포시 옥암동,0.49,89.14,9.39,165,전남,목포시,옥암동
87,전남 무안군 삼향읍,0.24,88.82,5.90,163,전남,무안군,삼향읍
94,전남 광양시 중동,0.37,98.09,4.78,115,전남,광양시,중동


In [18]:
광주 = df[df['시'] =="광주"]
광주

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
18,광주 북구 운암동,0.76,89.14,12.27,185,광주,북구,운암동
74,광주 남구 봉선동,0.43,91.30,3.90,358,광주,남구,봉선동


---

In [70]:
print("-----특목/자사고진학률 순위-----")
sorted_special_high

-----특목/자사고진학률 순위-----


,서울대입학률,4년제입학률,특목/자사고진학률
시,,,
충남,2.6,98.6,42.6
경북,4.0,100.0,20.6
서울,2.1,80.0,20.3
경남,0.9,83.2,14.5
대전,1.0,85.1,13.5
인천,1.5,76.5,13.0
부산,1.0,92.7,11.4
경기 성남시,2.3,85.8,10.9
경기 고양시,2.8,84.1,10.5


In [74]:
충남 = df[df['시'] =="충남"]
충남

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
61,충남 아산시 탕정면,2.65,98.59,42.59,29,충남,아산시,탕정면


### 충남 아산시 탕정면
- 삼성전자가 키운 동네. 
- 삼성덕분에 탕정미래초등학교, 탕정중학교, 충남외국어고등학교(특목고) 및 충남삼성고등학교(자사고)가 탄생
- 2022년 아산신도시 개발